### Load support for the %sql magic in Jupyter

In [ ]:
%load_ext sql

### Make sure your PostgreSQL server is running, and log into your database.

In [ ]:
%sql postgresql://vagrant:vagrant@:5432/optimizer
%sql SET search_path to sqlopt,public;

In [ ]:
import pandas as pd
result = %sql SELECT *, COALESCE(type, top_join) || ': ' || tables::text AS label\
                FROM opt_chosen_plan;
df = result.DataFrame()
df.to_json("notebooks/data/final.json", "records")
df

In [ ]:
from vega3 import Vega
Vega({
  "$schema": "https://vega.github.io/schema/vega/v3.0.json",
  "width": 600,
  "height": 200,
  "padding": 5,
  "signals": [
  ],
  "data": [
    {
      "name": "tree",
      "url": "data/final.json",
      "transform": [
        {
            "type": "formula",
            "expr": "'cost: ' + datum.cost",
            "as": "text2"
        },
        {
            "type": "formula",
            "expr": "'pgs: ' + datum.npages + ', tups: ' + datum.ntups",
            "as": "text3"
        },
           
        {
          "type": "joinaggregate",
          "fields": ["cost"],
          "ops": ["sum"],
          "as": ["total"]
        },
        {
          "type": "formula",
          "expr": "100*datum.cost/datum.total",
          "as": 'pct_cost'
        },
        {
          "type": "stratify",
          "key": "pid",
          "parentKey": "parent"
        },
        {
          "type": "tree",
          "method": "tidy",
          "nodeSize": [400,80],
          "size": [{"signal": "width"}, {"signal": "height"}],
          "as": ["x", "y", "depth", "children"],
          "sort": {"field": ["children", "sibling_id"], "order": ["descending", "ascending"]}
        }
      ]
    },
    {
      "name": "links",
      "source": "tree",
      "transform": [
        { "type": "treelinks", "key": "pid" },
        {
          "type": "linkpath",
          "orient": "horizontal",
          "shape": "line"
        }
      ]
    }
  ],
  "scales": [
    {
      "name": "color",
      "range": {"scheme": "orangered"},
      "domain": [0,100],
      "zero": "true"
    }
  ],
  "marks": [
    {
      "type": "path",
      "from": {"data": "links"},
      "encode": {
        "update": {
          "path": {"field": "path"},
          "stroke": {"value": "#ccc"}
        }
      }
    },
    {
      "type": "rect",
      "from": {"data": "tree"},
      "encode": {
        "enter": {
          "stroke": {"value": "#000"}, #"#fff"},
          "width": {"signal": "6*(datum.label.length + 10)"},
          "height": {"value": 60},
        },
        "update": {
          "x": {"field": "x", "offset": "-3*(datum.label.length+10)"},
          "y": {"field": "y", "offset": -10},
          "fill": {"scale": "color", "field": "pct_cost"},
        }
      }
    },
    {
      "type": "text",
      "from": {"data": "tree"},
      "encode": {
        "enter": {
          "text": {"field": "label"},
          "fontSize": {"value": 12},
          "baseline": {"value": "middle"},
          "x": {"field": "x"},
          "y": {"field": "y"},
          "align": {"value": "center"}
        }
      }
    },
    {
      "type": "text",
      "from": {"data": "tree"},
      "encode": {
        "enter": {
          "text": {"field": "text2"},
          "fontSize": {"value": 12},
          "baseline": {"value": "middle"},
          "x": {"field": "x"},
          "y": {"field": "y", "offset": 20},
          "align": {"value": "center"}
        }
      }
    },
    {
      "type": "text",
      "from": {"data": "tree"},
      "encode": {
        "enter": {
          "text": {"field": "text3"},
          "fontSize": {"value": 12},
          "baseline": {"value": "middle"},
          "x": {"field": "x"},
          "y": {"field": "y", "offset": 40},
          "align": {"value": "center"}
        }
      }
    }
  ]
})